In [1]:
!pip install beautifulsoup4
!pip install lxml

     |████████████████████████████████| 112kB 2.4MB/s eta 0:00:01
     |████████████████████████████████| 5.8MB 29.6MB/s eta 0:00:01


<h3> In the first step I'm importing required libraries including BeautifulSoup suggested in the Instruction.
    <br>Next step will scrap information in the table from the Wikipedia page.
    <br>Later on the new dataframe <b>toronto_df</b> will be created with all required columns and rows.
    <br>At the end I'm checking first 10 rows of scrapped table.</h3>

In [29]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import requests


wiki_html = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(wiki_html, 'html.parser')
table = soup.find('table', class_='wikitable sortable')
table_columns = []
for th_txt in table.tbody.findAll('th'):
    table_columns.append(th_txt.text.rstrip('\n'))

toronto_df = pd.DataFrame(columns=table_columns) 

for row in table.tbody.findAll('tr')[1:]:
    row_data = []
    for td_txt in row.findAll('td'):
        row_data.append(td_txt.text.rstrip('\n'))
    toronto_df = toronto_df.append({table_columns[0]: row_data[0],
                                    table_columns[1]: row_data[1],
                                    table_columns[2]: row_data[2]}, ignore_index=True)
toronto_df.head(10)

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor
7,M7A,Downtown Toronto,Queen's Park
8,M8A,Not assigned,Not assigned
9,M9A,Queen's Park,Not assigned


<h3> This part of the code will remove cells within Borough and Neighbourhood columns that are 'Not assigned' - as required in the Instruction

In [37]:
toronto_df.replace('Not assigned',np.nan, inplace=True)
toronto_df = toronto_df[toronto_df['Borough'].notnull()]
toronto_df.reset_index(drop=True, inplace=True)
toronto_df.Neighbourhood = np.where(toronto_df.Neighbourhood.isnull(),toronto_df.Borough,toronto_df.Neighbourhood)

toronto_df.head(10)

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,Lawrence Heights
4,M6A,North York,Lawrence Manor
5,M7A,Downtown Toronto,Queen's Park
6,M9A,Queen's Park,Queen's Park
7,M1B,Scarborough,Rouge
8,M1B,Scarborough,Malvern
9,M3B,North York,Don Mills North


<h3>Now it's possible to check number of rows in the dataframe. As we see it's <b>210</b>

In [38]:
toronto_df.count()

Postcode         210
Borough          210
Neighbourhood    210
dtype: int64